# KBL Predict - 2. Stacking

In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
RANDOM_SEED = 2022

## 10게임 데이터 전처리

In [2]:
# Colab google drive mount 경로

df_10game = pd.read_csv('/content/drive/MyDrive/KBL Prediction/dataset/recent_avg_record_10game.csv')

In [3]:
# win 컬럼 생성, 이기면 1 지면 0
df_10game['win'] = df_10game['score'] - df_10game['loss']
df_10game.loc[df_10game['score'] > df_10game['loss'], 'win'] = 1
df_10game.loc[df_10game['score'] < df_10game['loss'], 'win'] = 0

In [5]:
# X, y 분리
X = df_10game.iloc[:, 5:-1]
X.drop(columns=['ef','maxContiScoreCn', 'maxLeadScoreCn', 'playMin', 'playSec', 'scoreHighTimeCn', 'inout', 'inout1', 'idf', 'fb'], inplace=True)
y = df_10game.iloc[:,-1]

In [8]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_SEED)

In [9]:
# Standard Scale
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled = pd.DataFrame(data=X_train_scaled, columns=X_train.columns)
X_train_scaled

scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(data=X_test_scaled, columns=X_test.columns)

## 메타 모델에 필요한 학습데이터, 테스트 데이터 개별 모델로부터 가져오기

In [32]:
from sklearn.model_selection import KFold

def get_stacking_datasets(model, x_train_n, y_train_n, x_test_n, n_folds):
    # CV하기 위해 K-fold 설정
    x_train_n = np.array(x_train_n)
    y_train_n = np.array(y_train_n)
    x_test_n = np.array(x_test_n)
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 최종 메타 모델이 사용할 학습 데이터 반환을 위해서 넘파이 배열을 0으로 만들어서 초기화
    train_fold_pred = np.zeros((x_train_n.shape[0], 1)) # 2차원으로
    test_pred = np.zeros((x_test_n.shape[0], n_folds))
    print(model.__class__.__name__, '모델 시작')
    
    for folder_counter, (train_idx, valid_idx) in enumerate(kf.split(x_train_n)):
        # 개별 모델 내부에서 학습하고 1개의 fold로 예측할 데이터 셋 추출
        print(f" Fold 횟수 : {folder_counter+1}")
        x_tr = x_train_n[train_idx]
        y_tr = y_train_n[train_idx]
        x_te = x_train_n[valid_idx]
        
        # 개별 모델이 학습한 후 1개의 fold데이터셋으로 예측값 반환 후 최종 메타모델이 학습할 데이터셋에 첨가
        model.fit(x_tr, y_tr)
        train_fold_pred[valid_idx, :] = model.predict(x_te).reshape(-1,1)
        # 개별 모델이 원본 데이터셋의 검증 데이터셋을 기반으로 예측 결과값 반환 후 최종 메타모델이 검증할 데이터셋에 첨가
        test_pred[:, folder_counter] = model.predict(x_test_n)
    
    # 개별모델안에서 테스트 데이터셋을 기반으로 예측한 결과값들 mean취해주고 2차원으로 바꾸어주기
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean

## 이전에 Grid search로 찾은 hyper parameter로 모델 생성

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression # 메타 모델

rf = RandomForestClassifier(max_depth=8, min_samples_leaf=6, min_samples_split=2, n_estimators=250, n_jobs=-1, random_state=RANDOM_SEED)
gb = GradientBoostingClassifier(learning_rate=0.07, max_depth=3, min_samples_leaf=8, n_estimators=100, random_state=RANDOM_SEED)
ada = AdaBoostClassifier(learning_rate=0.05, n_estimators=300, random_state=RANDOM_SEED)
lr = LogisticRegression()

## 메타모델 실행

In [34]:
from sklearn.metrics import accuracy_score

# 개별 모델로부터 메타 모델에 필요한 데이터 셋 만들기
rf_train, rf_test = get_stacking_datasets(rf, X_train_scaled, y_train, X_test_scaled, 7)
gb_train, gb_test = get_stacking_datasets(gb, X_train_scaled, y_train, X_test_scaled, 7)
ada_train, ada_test = get_stacking_datasets(ada, X_train_scaled, y_train, X_test_scaled, 7)

# 개별모델이 생성한 학습/검증 데이터 최종 메타 모델이 학습/검증하도록 결합
stack_final_x_train = np.concatenate((rf_train, gb_train, ada_train), axis=1)
stack_final_x_test = np.concatenate((rf_test, gb_test, ada_test), axis=1)

# 최종 메타모델로 학습
# 최종 메타모델 학습시 label은 원본 데이터의 label(y값)
lr.fit(stack_final_x_train, y_train)
stack_final_pred = lr.predict(stack_final_x_test)

# 최종 메타모델 성능 평가(비교할 때 원본 데이터의 검증 데이터 label과 비교)
print(f"최종 메타모델 정확도 : {accuracy_score(y_test, stack_final_pred):.4f}")

RandomForestClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
 Fold 횟수 : 6
 Fold 횟수 : 7
GradientBoostingClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
 Fold 횟수 : 6
 Fold 횟수 : 7
AdaBoostClassifier 모델 시작
 Fold 횟수 : 1
 Fold 횟수 : 2
 Fold 횟수 : 3
 Fold 횟수 : 4
 Fold 횟수 : 5
 Fold 횟수 : 6
 Fold 횟수 : 7
최종 메타모델 정확도 : 0.5996
